In [2]:
from pyspark.sql.functions import *
from pyspark.sql.types import *


In [17]:
!pip install pyreadline

     |████████████████████████████████| 112kB 1.4MB/s eta 0:00:01
  Running setup.py install for pyreadline ... done


In [3]:
employees = spark.read.csv("HR_employess.csv",header=True)
employees = employees.withColumn("SALARY",employees["SALARY"].cast(DoubleType()))


In [30]:
departments = spark.read.csv("HR_departments.csv",header=True)
departments.show()

+-------------+--------------------+----------+-----------+
|DEPARTMENT_ID|     DEPARTMENT_NAME|MANAGER_ID|LOCATION_ID|
+-------------+--------------------+----------+-----------+
|           10|      Administration|       200|       1700|
|           20|           Marketing|       201|       1800|
|           30|          Purchasing|       114|       1700|
|           40|     Human Resources|       203|       2400|
|           50|            Shipping|       121|       1500|
|           60|                  IT|       103|       1400|
|           70|    Public Relations|       204|       2700|
|           80|               Sales|       145|       2500|
|           90|           Executive|       100|       1700|
|          100|             Finance|       108|       1700|
|          110|          Accounting|       205|       1700|
|          120|            Treasury|      null|       1700|
|          130|       Corporate Tax|      null|       1700|
|          140|  Control And Credit|    

In [18]:
employees.show(5)

+-----------+----------+---------+--------+------------+-------------------+-------+-------+--------------+----------+-------------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER|          HIRE_DATE| JOB_ID| SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|
+-----------+----------+---------+--------+------------+-------------------+-------+-------+--------------+----------+-------------+
|        100|    Steven|     King|   SKING|515.123.4567|1987-06-17 00:00:00|AD_PRES|24000.0|          null|      null|           90|
|        101|     Neena|  Kochhar|NKOCHHAR|515.123.4568|1989-09-21 00:00:00|  AD_VP|17000.0|          null|       100|           90|
|        102|       Lex|  De Haan| LDEHAAN|515.123.4569|1993-01-13 00:00:00|  AD_VP|17000.0|          null|       100|           90|
|        103| Alexander|   Hunold| AHUNOLD|590.423.4567|1990-01-03 00:00:00|IT_PROG| 9000.0|          null|       102|           60|
|        104|     Bruce|    Ernst|  BERNST|590.423.4568|1991-05-21 00

In [5]:
employees.printSchema()

root
 |-- EMPLOYEE_ID: string (nullable = true)
 |-- FIRST_NAME: string (nullable = true)
 |-- LAST_NAME: string (nullable = true)
 |-- EMAIL: string (nullable = true)
 |-- PHONE_NUMBER: string (nullable = true)
 |-- HIRE_DATE: string (nullable = true)
 |-- JOB_ID: string (nullable = true)
 |-- SALARY: double (nullable = true)
 |-- COMMISSION_PCT: string (nullable = true)
 |-- MANAGER_ID: string (nullable = true)
 |-- DEPARTMENT_ID: string (nullable = true)



In [69]:
locations = spark.read.option("ignoreLeadingWhiteSpace", True).option("ignoreTrailingWhiteSpace", True).csv("HR_locations.csv",header=True)
locations.registerTempTable("locations")
locations.show()
locations.printSchema()


+-----------+--------------------+-----------+-------------------+----------------+----------+
|location_id|      street_address|postal_code|               city|  state_province|country_id|
+-----------+--------------------+-----------+-------------------+----------------+----------+
|       1000|1297 Via Cola di Rie|      00989|               Roma|            null|        IT|
|       1100|93091 Calle della...|      10934|             Venice|            null|        IT|
|       1200|    2017 Shinjuku-ku|       1689|              Tokyo|Tokyo Prefecture|        JP|
|       1300|     9450 Kamiya-cho|       6823|          Hiroshima|            null|        JP|
|       1400| 2014 Jabberwocky Rd|      26192|          Southlake|           Texas|        US|
|       1500| 2011 Interiors Blvd|      99236|South San Francisco|      California|        US|
|       1600|      2007 Zagora St|      50090|    South Brunswick|      New Jersey|        US|
|       1700|     2004 Charade Rd|      98199|    

In [22]:
employees.registerTempTable("employees")

In [23]:
departments.registerTempTable("departments")

In [24]:
spark.sql("select * from employees").show(2)

+-----------+----------+---------+--------+------------+-------------------+-------+-------+--------------+----------+-------------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER|          HIRE_DATE| JOB_ID| SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|
+-----------+----------+---------+--------+------------+-------------------+-------+-------+--------------+----------+-------------+
|        100|    Steven|     King|   SKING|515.123.4567|1987-06-17 00:00:00|AD_PRES|24000.0|          null|      null|           90|
|        101|     Neena|  Kochhar|NKOCHHAR|515.123.4568|1989-09-21 00:00:00|  AD_VP|17000.0|          null|       100|           90|
+-----------+----------+---------+--------+------------+-------------------+-------+-------+--------------+----------+-------------+
only showing top 2 rows



In [25]:
# 1. Write a query to find the name (first_name, last_name) 
# and the salary of the employees who have a higher salary than the employee whose last_name='Bull'.

spark.sql("select FIRST_NAME,last_name,salary from employees where salary > \
(select salary from employees where last_name='Bull')").show(2)

+----------+---------+-------+
|FIRST_NAME|last_name| salary|
+----------+---------+-------+
|    Steven|     King|24000.0|
|     Neena|  Kochhar|17000.0|
+----------+---------+-------+
only showing top 2 rows



In [26]:
bull_lat_name = employees.filter(col('last_name') == 'Bull').select('salary').rdd.map(lambda x:x.salary).collect()
bull_lat_name

[4100.0]

In [27]:
employees.filter(col('salary')> bull_lat_name[0]).select('FIRST_NAME','last_name','salary').show()

+-----------+---------+-------+
| FIRST_NAME|last_name| salary|
+-----------+---------+-------+
|     Steven|     King|24000.0|
|      Neena|  Kochhar|17000.0|
|        Lex|  De Haan|17000.0|
|  Alexander|   Hunold| 9000.0|
|      Bruce|    Ernst| 6000.0|
|      David|   Austin| 4800.0|
|      Valli|Pataballa| 4800.0|
|      Diana|  Lorentz| 4200.0|
|      Nancy|Greenberg|12000.0|
|     Daniel|   Faviet| 9000.0|
|       John|     Chen| 8200.0|
|     Ismael|  Sciarra| 7700.0|
|Jose Manuel|    Urman| 7800.0|
|       Luis|     Popp| 6900.0|
|        Den| Raphaely|11000.0|
|    Matthew|    Weiss| 8000.0|
|       Adam|    Fripp| 8200.0|
|      Payam| Kaufling| 7900.0|
|     Shanta|  Vollman| 6500.0|
|      Kevin|  Mourgos| 5800.0|
+-----------+---------+-------+
only showing top 20 rows



In [49]:
# 2. Write a query to find the name (first_name, last_name) 
# of all employees who works in the IT department.
# spark.sql("select DEPARTMENT_ID from departments where DEPARTMENT_NAME ='IT'").show()
spark.sql("select first_name ,last_name from employees where DEPARTMENT_ID in \
          (select DEPARTMENT_ID from departments where DEPARTMENT_NAME ='IT') ").show()


+----------+---------+
|first_name|last_name|
+----------+---------+
| Alexander|   Hunold|
|     Bruce|    Ernst|
|     David|   Austin|
|     Valli|Pataballa|
|     Diana|  Lorentz|
+----------+---------+



In [84]:
# 3. Write a query to find the name (first_name, last_name) 
# of the employees who have a manager and worked in a USA based department.
x = spark.sql("select emp.first_name ,emp.last_name from employees emp join  \
                            departments dep on emp.DEPARTMENT_ID = dep.DEPARTMENT_ID \
                            join locations loc on loc.location_id = dep.location_id \
                            where dep.MANAGER_ID != 0 and loc.country_id == 'US' ")

In [ ]:
x = spark.sql(""" 
select first_name , last_name from employees where
    DEPARTMENT_ID in (select DEPARTMENT_ID from departments where MANAGER_ID!=0
    and location_id in (select location_id from  locations where country_id == 'US') )
    """)
x.explain() # Both the joins and sub queries use the same plan for query execution

In [89]:
# 4. Write a query to find the name (first_name, last_name) of the employees who are managers.
x = spark.sql("""
                select first_name , last_name from employees where EMPLOYEE_ID in (
                select MANAGER_ID from employees )""").show()

+----------+---------+
|first_name|last_name|
+----------+---------+
|    Steven|     King|
|     Neena|  Kochhar|
|       Lex|  De Haan|
| Alexander|   Hunold|
|     Nancy|Greenberg|
|       Den| Raphaely|
|   Matthew|    Weiss|
|      Adam|    Fripp|
|     Payam| Kaufling|
|    Shanta|  Vollman|
|     Kevin|  Mourgos|
|      John|  Russell|
|     Karen| Partners|
|   Alberto|Errazuriz|
|    Gerald|Cambrault|
|     Eleni|  Zlotkey|
|   Michael|Hartstein|
|   Shelley|  Higgins|
+----------+---------+



In [93]:
# 5. Write a query to find the name (first_name, last_name),
# and salary of the employees whose salary is greater than the average salary. 
x = spark.sql("""
                select first_name , last_name,salary from employees where salary > (
                select avg(salary) from employees )
                """).show()


+-----------+---------+-------+
| first_name|last_name| salary|
+-----------+---------+-------+
|     Steven|     King|24000.0|
|      Neena|  Kochhar|17000.0|
|        Lex|  De Haan|17000.0|
|  Alexander|   Hunold| 9000.0|
|      Nancy|Greenberg|12000.0|
|     Daniel|   Faviet| 9000.0|
|       John|     Chen| 8200.0|
|     Ismael|  Sciarra| 7700.0|
|Jose Manuel|    Urman| 7800.0|
|       Luis|     Popp| 6900.0|
|        Den| Raphaely|11000.0|
|    Matthew|    Weiss| 8000.0|
|       Adam|    Fripp| 8200.0|
|      Payam| Kaufling| 7900.0|
|     Shanta|  Vollman| 6500.0|
|       John|  Russell|14000.0|
|      Karen| Partners|13500.0|
|    Alberto|Errazuriz|12000.0|
|     Gerald|Cambrault|11000.0|
|      Eleni|  Zlotkey|10500.0|
+-----------+---------+-------+
only showing top 20 rows



In [ ]:
# 6. Write a query to find the name (first_name, last_name), 
# and salary of the employees whose salary is equal to the minimum salary for their job grade.

x = spark.sql("""
                select first_name , last_name,salary from employees where salary == (
                select min(salary) from employees group by  )
                """).show()
